# ANSWERING BUSINESS QUESTIONS USING THE CHINOOK DATABASE
This project is part of the guided projects available on [Dataquest.io](https://dataquest.io). The [Chinook](https://github.com/lerocha/chinook-database) database contains information about a fictional digital music shop - kind of like a mini-iTunes store. The project aim is to get answers on business-like questions that might be asked IRL.

**Database Schema**
![Chinook.db Schema](https://s3.amazonaws.com/dq-content/189/chinook-schema.svg "The database schema")

## 1. Connecting to the Database
Cells with code to connect to the database, and inspecting the database tables.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,album,album,2,"CREATE TABLE [album]( [album_id] INTEGER PRIMARY KEY NOT NULL, [title] NVARCHAR(160) NOT NULL, [artist_id] INTEGER NOT NULL, FOREIGN KEY ([artist_id]) REFERENCES [artist] ([artist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,artist,artist,3,"CREATE TABLE [artist]( [artist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,customer,customer,4,"CREATE TABLE [customer]( [customer_id] INTEGER PRIMARY KEY NOT NULL, [first_name] NVARCHAR(40) NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [company] NVARCHAR(80), [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60) NOT NULL, [support_rep_id] INTEGER, FOREIGN KEY ([support_rep_id]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,employee,employee,5,"CREATE TABLE [employee]( [employee_id] INTEGER PRIMARY KEY NOT NULL, [last_name] NVARCHAR(20) NOT NULL, [first_name] NVARCHAR(20) NOT NULL, [title] NVARCHAR(30), [reports_to] INTEGER, [birthdate] DATETIME, [hire_date] DATETIME, [address] NVARCHAR(70), [city] NVARCHAR(40), [state] NVARCHAR(40), [country] NVARCHAR(40), [postal_code] NVARCHAR(10), [phone] NVARCHAR(24), [fax] NVARCHAR(24), [email] NVARCHAR(60), FOREIGN KEY ([reports_to]) REFERENCES [employee] ([employee_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,genre,genre,6,"CREATE TABLE [genre]( [genre_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,invoice,invoice,7,"CREATE TABLE [invoice]( [invoice_id] INTEGER PRIMARY KEY NOT NULL, [customer_id] INTEGER NOT NULL, [invoice_date] DATETIME NOT NULL, [billing_address] NVARCHAR(70), [billing_city] NVARCHAR(40), [billing_state] NVARCHAR(40), [billing_country] NVARCHAR(40), [billing_postal_code] NVARCHAR(10), [total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([customer_id]) REFERENCES [customer] ([customer_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,invoice_line,invoice_line,8,"CREATE TABLE [invoice_line]( [invoice_line_id] INTEGER PRIMARY KEY NOT NULL, [invoice_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, [unit_price] NUMERIC(10,2) NOT NULL, [quantity] INTEGER NOT NULL, FOREIGN KEY ([invoice_id]) REFERENCES [invoice] ([invoice_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"
table,media_type,media_type,9,"CREATE TABLE [media_type]( [media_type_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist,playlist,10,"CREATE TABLE [playlist]( [playlist_id] INTEGER PRIMARY KEY NOT NULL, [name] NVARCHAR(120))"
table,playlist_track,playlist_track,11,"CREATE TABLE [playlist_track]( [playlist_id] INTEGER NOT NULL, [track_id] INTEGER NOT NULL, CONSTRAINT [pk_playlist_track] PRIMARY KEY ([playlist_id], [track_id]), FOREIGN KEY ([playlist_id]) REFERENCES [playlist] ([playlist_id]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([track_id]) REFERENCES [track] ([track_id]) ON DELETE NO ACTION ON UPDATE NO ACTION)"


## 2. Selecting Albums to Purchase
The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now – we have the artist names, and the genre of music they produce:

|Artist Name|Genre|
|-----|-------|
|Regal|Hip-Hop|
|Red Tone|Punk|
|Meteor and the Girls|Pop|
|Slim Jim Bites|Blues|

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [3]:
%%sql
WITH usa_customers_tracks AS
    (
    SELECT il.*
    FROM invoice_line AS il
    INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
    WHERE c.country = "USA"
    )

SELECT 
    g.name AS genre,
    SUM(uct.quantity) AS sold_tracks,
    ROUND(
        CAST(COUNT(uct.quantity) AS FLOAT) / 
            (SELECT COUNT(*)
             FROM usa_customers_tracks
            ) * 100, 
        2) AS percentage_sold
    FROM usa_customers_tracks AS uct
    INNER JOIN track AS t ON t.track_id = uct.track_id
    INNER JOIN genre AS g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook.db
Done.


genre,sold_tracks,percentage_sold
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Pop,22,2.09
Latin,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


Based on the genre sales in USA I would recommend following three artists:
1. Red Tone, Punk
2. Slim Jim Bites,Blues
3. Meteor and the Girls, Pop

Selected three generes makes 17.89% of the total USA sales. The best genre where to invest is Rock music, with 53.38% of USA market

## 3. Analyzing Employee Performance
Each customer for the Chinook store gets assigned to a sales support agent within the company when they first make a purchase. You have been asked to analyze the purchases of customers belonging to each employee to see if any sales support agent is performing either better or worse than the others.

In [12]:
%%sql

WITH sales_rep AS
    (
    SELECT 
        e.employee_id,
        e.first_name || " " || e.last_name AS employee_name,
        e.hire_date,
        e.birthdate,
        e.title,
        ROUND(SUM(i.total), 2) AS total_sales
        FROM employee AS e
        LEFT JOIN customer AS c ON c.support_rep_id = e.employee_id
        LEFT JOIN invoice AS i ON i.customer_id = c.customer_id
    WHERE e.title = "Sales Support Agent"
    GROUP BY 1
    )

SELECT *
FROM sales_rep
ORDER BY total_sales DESC;

 * sqlite:///chinook.db
Done.


employee_id,employee_name,hire_date,birthdate,title,total_sales
3,Jane Peacock,2017-04-01 00:00:00,1973-08-29 00:00:00,Sales Support Agent,1731.51
4,Margaret Park,2017-05-03 00:00:00,1947-09-19 00:00:00,Sales Support Agent,1584.0
5,Steve Johnson,2017-10-17 00:00:00,1965-03-03 00:00:00,Sales Support Agent,1393.92


Average total sales per Sales Suppoert Agent are 1569.81. The best performing agent is Jane Peacock, beating the average sales by 10.3%. The lowest performing agent is Steve Johnson, underperfoming by 11.2%. It seems like the age of the agents does not play a role, however the hire date probably does. The most senior employees are performing better than juniors.

## 4. Analyzing Sales by Country
Next task is to analyze the sales data for customers from each different country. You have been given guidance to use the `country` value from the `customers` table, and ignore the `country` from the billing address in the `invoice` table.

Calculate data, for each country, on the:
- total number of customers
- total value of sales
- average value of sales per customer
- average order value

In [14]:
%%sql
WITH country_sales AS
(
    SELECT
        CASE
            WHEN (
                  SELECT COUNT(*)
                  FROM customer
                  WHERE country = c.country
                  ) = 1 THEN "Other"
            ELSE c.country   
        END AS country,
    c.customer_id,
    il.*
    FROM invoice_line AS il
    INNER JOIN invoice AS i ON i.invoice_id = il.invoice_id
    INNER JOIN customer AS c ON c.customer_id = i.customer_id
)

SELECT
    country,
    COUNT(distinct customer_id) AS total_customers,
    ROUND(SUM(unit_price), 2) AS total_sales,
    ROUND(SUM(unit_price) / COUNT(distinct customer_id), 2) AS customer_lifetime_value,
    ROUND(SUM(unit_price) / COUNT(DISTINCT invoice_id), 2) AS average_order
FROM (
     SELECT
        cs.*,
        CASE
            WHEN cs.country = "Other" THEN 1
            ELSE 0
        END AS Sort
      FROM country_sales AS cs
     )
GROUP BY country
ORDER BY sort ASC, total_sales DESC;

 * sqlite:///chinook.db
Done.


country,total_customers,total_sales,customer_lifetime_value,average_order
USA,13,1040.49,80.04,7.94
Canada,8,535.59,66.95,7.05
Brazil,5,427.68,85.54,7.01
France,5,389.07,77.81,7.78
Germany,4,334.62,83.66,8.16
Czech Republic,2,273.24,136.62,9.11
United Kingdom,3,245.52,81.84,8.77
Portugal,2,185.13,92.57,6.38
India,2,183.15,91.58,8.72
Other,15,1094.94,73.0,7.45


Based on the data there might be an opportunity in following countries:
- Czech Republic
- Portugal
- India
- India
- Germany

The sales are the highest in Northern America. There is an opportunity to boost sells in UK and India because both countries have strong base of English Speakers – the marketing materials can be somewhat the same as in Northern America. Czech Republic have the hgihtest customer lifetime value and average order - can be potentionaly market with good ROI. Keep in mind that the base data is relatively small sample. Some countries having only two customers.

## 5. Albums vs Individual Tracks
The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:
- purchase a whole album,
- purchase a collection of one or more individual tracks.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

In [15]:
%%sql
WITH invoice_track AS
    (
    SELECT
        il.invoice_id AS invoice_id,
        MIN(il.track_id) AS first_track_id
    FROM invoice_line il
    GROUP BY 1
)

SELECT
    album_purchase,
    COUNT(invoice_id) AS number_of_invoices,
    ROUND(CAST(count(invoice_id) AS FLOAT) / (
                                         SELECT COUNT(*) FROM invoice
                                      ) * 100, 1) AS percent
FROM
(
    SELECT
        it.*,
        CASE
            WHEN
                 (
                  SELECT t.track_id 
                  FROM track AS t
                  WHERE t.album_id = (
                                      SELECT t2.album_id 
                                      FROM track AS t2
                                      WHERE t2.track_id = it.first_track_id
                                     ) 

                  EXCEPT 

                  SELECT il2.track_id 
                  FROM invoice_line AS il2
                  WHERE il2.invoice_id = it.invoice_id
                 ) IS NULL
             AND
                 (
                  SELECT il2.track_id 
                  FROM invoice_line AS il2
                  WHERE il2.invoice_id = it.invoice_id

                  EXCEPT 

                  SELECT t.track_id 
                  FROM track AS t
                  WHERE t.album_id = (
                                      SELECT t2.album_id 
                                      FROM track AS t2
                                      WHERE t2.track_id = it.first_track_id
                                     ) 
                 ) IS NULL
             THEN "yes"
             ELSE "no"
         END AS "album_purchase"
     FROM invoice_track it
)
GROUP BY album_purchase;

 * sqlite:///chinook.db
Done.


album_purchase,number_of_invoices,percent
no,500,81.4
yes,114,18.6


Album purchases consists of 18.6% of the company purchases. When buying only single tracks the company would loose one fifth of the sales.